In [1]:
from datetime import timedelta
import os

from dateutil import tz
import numpy as np
import pandas as pd

from tqdm import tqdm

import sys
sys.path.insert(0, 'C:/Users/gioef/Desktop/onnela_lab/forest')
from forest.oak.base import preprocess_dates, preprocess_bout


KeyboardInterrupt



In [ ]:
beiwe_ids = ["z4qz8tqn", "x7fan661", "1z71l8a5", "vkuj9c7n", "2he8nebs", "ilp5da7y", "pyl4f2q3", "wzbz6rti", "x6lsyjql", "3gg4zvbm",
            "k7h4hqy4", "jj1p8ov2", "bq2owy9n", "888ferxx", "k6ammi28", "k6ammi28", "vzw1eqkd", "yfntte3n", "z4qz8tqn", "cwi3ijj1",
            "ztomyxfb", "l5ca9gyk"]
time_start = "2024-03-26 00_00_00"
time_end = "2024-06-26 00_00_00"
study_folder = 'adolescent_bipolar_study/data'
tz_str = 'America/New_York'
# determine timezone shift
fmt = '%Y-%m-%d %H_%M_%S'
from_zone = tz.gettz('UTC')
to_zone = tz.gettz(tz_str) if tz_str else from_zone

In [ ]:
for _, user in tqdm(enumerate(beiwe_ids), desc="Users", total=len(beiwe_ids)):


    # get file list
    source_folder = os.path.join(study_folder, user, "accelerometer")
    file_list = os.listdir(source_folder)
    file_list.sort()

    dates_shifted, date_start, date_end = preprocess_dates(
        file_list, time_start, time_end, fmt, from_zone, to_zone
    )

    days = pd.date_range(date_start, date_end, freq='D')

    # allocate memory
    t_ind_pydate = pd.Series([], dtype='datetime64[ns]')
    t_ind_pydate_str = None

    freq = 'T'

    days_hourly = pd.date_range(date_start, date_end+timedelta(days=1),
                                freq=freq)[:-1]

    t_ind_pydate = days_hourly.to_pydatetime()
    t_ind_pydate_str = t_ind_pydate.astype(str)

    # create a dataframe with 2 columns: timestamp, vm
    res = pd.DataFrame({'timestamp': [], 'vm': []})

    for d_ind, d_datetime in tqdm(enumerate(days), desc="Days", total=len(days)):
        # find file indices for this d_ind
        file_ind = [i for i, x in enumerate(dates_shifted)
                    if x == d_datetime]
        # check if there is at least one file for a given day
        if len(file_ind) <= 0:
            print("No files for this day.")
            continue
        # initiate dataframe
        data = pd.DataFrame()
        # load data for a given day
        for f in file_ind:
            # print("File: %d", f)
            # read data
            file_path = os.path.join(source_folder, file_list[f])
            data = pd.concat([data, pd.read_csv(file_path)], axis=0)

        # extract data
        timestamp = np.array(data["timestamp"]) / 1000
        x = np.array(data["x"], dtype="float64")  # x-axis acc.
        y = np.array(data["y"], dtype="float64")  # y-axis acc.
        z = np.array(data["z"], dtype="float64")  # z-axis acc.
        # preprocess data fragment
        t_bout_interp, vm_bout = preprocess_bout(timestamp, x, y, z)
        if len(t_bout_interp) == 0:  # no valid data to process here
            continue

        vm_bout = vm_bout[::600]  # downsample to 1 observation per minute
        t_bout_interp = t_bout_interp[::60]  # downsample to 1 observation per minute

        assert len(vm_bout) == len(t_bout_interp)

        # append to the result
        res = pd.concat([res, pd.DataFrame({'timestamp': t_bout_interp, 'vm': vm_bout})], axis=0)

    # create a new column with the datetime from the timestamp using time zone
    res['datetime'] = pd.to_datetime(res['timestamp'], unit='s')
    # convert to the desired time zone
    res['datetime'] = res['datetime'].dt.tz_localize('UTC').dt.tz_convert(tz_str)
    res['date'] = res['datetime'].dt.date 
    res['day_of_week'] = res['datetime'].dt.dayofweek # Monday=0, Sunday=6
    res['hour'] = res['datetime'].dt.hour
    res['minute'] = res['datetime'].dt.minute

    # save to csv
    res.to_csv(f'adolescent_bipolar_study/processed_data/{user}.csv', index=False)
